In [89]:
# !pip install ccxt
# !pip install slacker
import ccxt
import pprint
from datetime import datetime
from slacker import Slacker
import time

In [2]:
# 파일로부터 apiKey, Secret 읽기 
with open("apiKey_ham.txt") as f:
    lines = f.readlines()
    apiKey = lines[0].strip() 
    secret = lines[1].strip() 
    slackToken = lines[2].strip()

In [3]:
slack = Slacker(slackToken)
binance = ccxt.binance()

In [ ]:
# 슬랙 메세지 보내기
# slack.chat.post_message('#autostock','hi bum!')

In [ ]:
# 티커들 조회
markets = binance.fetch_tickers()
print(markets.keys())

In [84]:
# 현재가 조회
ticker = binance.fetch_ticker('ETH/USDT')
print(ticker['open'], ticker['high'], ticker['low'], ticker['close'])

3854.69 3925.95 3752.0 3796.35


In [20]:
# 과거 데이터 조회
ohlcvs = binance.fetch_ohlcv('ETH/USDT')
# for ohlc in ohlcvs:
#     print(datetime.fromtimestamp(ohlc[0]/1000).strftime('%Y-%m-%d %H:%M:%S'))

---

In [4]:
# 선물 잔고 조회
# free : 보유중, used : 거래진행중, 
binance = ccxt.binance({
    'apiKey': apiKey,
    'secret': secret,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'future'
    }
})
balance = binance.fetch_balance(params={"type": "future"})
print(balance['USDT'])

{'free': 80.95354954, 'used': 0.0, 'total': 80.95354954}


---

In [6]:
def showDateTime(dateTime):
    return datetime.fromtimestamp(dateTime/1000).strftime('%Y-%m-%d %H:%M:%S')

In [46]:
# ohlcv
eth5m = binance.fetch_ohlcv(
    symbol="ETH/USDT", 
    timeframe='5m', 
    since=None, 
    limit=20)
for e in eth5m:
    print(showDateTime(e[0]), e[1:])

2021-10-20 00:10:00 [3800.9, 3812.97, 3799.22, 3806.68, 4744.012]
2021-10-20 00:15:00 [3806.69, 3818.84, 3803.41, 3808.0, 8986.006]
2021-10-20 00:20:00 [3808.14, 3813.78, 3780.0, 3799.48, 27808.379]
2021-10-20 00:25:00 [3799.48, 3805.57, 3794.61, 3799.14, 5413.68]
2021-10-20 00:30:00 [3799.14, 3803.56, 3794.73, 3798.72, 3958.311]
2021-10-20 00:35:00 [3798.72, 3801.14, 3796.42, 3799.06, 2434.759]
2021-10-20 00:40:00 [3799.23, 3802.61, 3798.21, 3800.09, 1906.846]
2021-10-20 00:45:00 [3800.09, 3800.46, 3787.83, 3794.0, 5032.867]
2021-10-20 00:50:00 [3794.0, 3796.47, 3790.0, 3793.06, 2782.586]
2021-10-20 00:55:00 [3793.05, 3796.79, 3775.2, 3776.43, 8877.966]
2021-10-20 01:00:00 [3776.51, 3785.88, 3776.5, 3779.01, 8521.054]
2021-10-20 01:05:00 [3779.01, 3788.6, 3778.61, 3785.14, 3761.961]
2021-10-20 01:10:00 [3785.13, 3790.57, 3780.01, 3781.77, 4904.484]
2021-10-20 01:15:00 [3781.73, 3789.35, 3777.62, 3788.56, 3016.655]
2021-10-20 01:20:00 [3788.55, 3794.0, 3787.9, 3793.33, 5022.619]
2021-1

In [112]:
def checkIfGoodToBuy():
    eth5m = binance.fetch_ohlcv(
        symbol="ETH/USDT",
        timeframe='5m',
        since=None,
        limit=10)
    candleL = []
    for e in eth5m:
        candle = e[4] - e[1]
        if candle > 3:
            candleL.append("white")
        elif candle < -3:
            candleL.append("black")
        else:
            candleL.append("")
    print(candleL)
    if len(candleL) < 3:
        return False
    goodStack = 0
    for i in range(len(candleL) - 1, -1, -1):
        if candleL[i] == "white":
            return False
        elif candleL[i] == "black":
            goodStack += 1
        elif candleL[i] == "" and goodStack >= 3:
            return True
        if goodStack >= 4:
            return True
    return False
checkIfGoodToBuy()

['', 'black', 'black', '', 'black', 'white', '', '', 'black', '']


False

---

In [33]:
# 매수 매도
order = binance.create_market_buy_order(
    symbol="ETH/USDT",
    amount=0.002, 
)
pprint.pprint(order)
 
# order = binance.create_market_sell_order(
#     symbol="ETH/USDT",
#     amount=0.002, 
# )
# pprint.pprint(order)


{'amount': 0.002,
 'average': 3849.86,
 'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
 'cost': 7.69972,
 'datetime': None,
 'fee': None,
 'fees': [],
 'filled': 0.002,
 'id': '8389765509100861083',
 'info': {'avgPrice': '3849.86000',
          'clientOrderId': 'x-xcKtGhcu27a24d72355b4a0c810ef1',
          'closePosition': False,
          'cumQty': '0.002',
          'cumQuote': '7.69972',
          'executedQty': '0.002',
          'orderId': '8389765509100861083',
          'origQty': '0.002',
          'origType': 'MARKET',
          'positionSide': 'BOTH',
          'price': '0',
          'priceProtect': False,
          'reduceOnly': False,
          'side': 'BUY',
          'status': 'FILLED',
          'stopPrice': '0',
          'symbol': 'ETHUSDT',
          'timeInForce': 'GTC',
          'type': 'MARKET',
          'updateTime': '1634482056016',
          'workingType': 'CONTRACT_PRICE'},
 'lastTradeTimestamp': None,
 'postOnly': False,
 'price': 3849.86,
 'remaining

In [51]:
def getTodayDate():
    now = datetime.now()
    nowDate = now.strftime('%Y%m%d')
    return nowDate

'20211020'

In [66]:

price = 3849.86
updateTime = '1634482056016'

fileName = getTodayDate()+".txt"
try:
    with open(fileName) as f:
        lines = f.readlines()
except:
    f = open(fileName, "w")
    f.close()
    lines = []
lines.append(updateTime + "\n")
with open(fileName, "w") as f:
    for line in lines:
        f.write(line)


In [99]:
# 현재 포지션 확인
balance = binance.fetch_balance()
position = [x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]
print(position)
print(float(position["positionAmt"]) > 0)
print(position["entryPrice"])

{'symbol': 'ETHUSDT', 'initialMargin': '0', 'maintMargin': '0', 'unrealizedProfit': '0.00000000', 'positionInitialMargin': '0', 'openOrderInitialMargin': '0', 'leverage': '10', 'isolated': True, 'entryPrice': '0.0', 'maxNotional': '4000000', 'positionSide': 'BOTH', 'positionAmt': '0.000', 'notional': '0', 'isolatedWallet': '0', 'updateTime': '1634491502625'}
False
0.0


In [101]:
balance = binance.fetch_balance()
entryPrice = float([x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]["entryPrice"])
entryPrice

0.0

In [104]:

balance = binance.fetch_balance()
float([x for x in balance['info']['positions'] if x['symbol'] == "ETHUSDT"][0]["positionAmt"])

0.0

In [83]:
binance.trades

{}

In [79]:
datetime.now()

datetime.datetime(2021, 10, 20, 2, 22, 47, 919297)

In [60]:
# 현재 포지션 2퍼 위에 매도 예약
# targetPrice = round(float(position["entryPrice"]) * 1.0022, 2)
# print(targetPrice)
# binance.create_limit_sell_order(
#     symbol = 'ETH/USDT',
#     amount = 0.002,
#     price = targetPrice
# )

3858.33


{'info': {'orderId': '8389765509101346200',
  'symbol': 'ETHUSDT',
  'status': 'NEW',
  'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
  'price': '3858.33',
  'avgPrice': '0.00000',
  'origQty': '0.002',
  'executedQty': '0',
  'cumQty': '0',
  'cumQuote': '0',
  'timeInForce': 'GTC',
  'type': 'LIMIT',
  'reduceOnly': False,
  'closePosition': False,
  'side': 'SELL',
  'positionSide': 'BOTH',
  'stopPrice': '0',
  'workingType': 'CONTRACT_PRICE',
  'priceProtect': False,
  'origType': 'LIMIT',
  'updateTime': '1634483299501'},
 'id': '8389765509101346200',
 'clientOrderId': 'x-xcKtGhcu41680a6da6274810a76ddb',
 'timestamp': 1634483299501,
 'datetime': '2021-10-17T15:08:19.501Z',
 'lastTradeTimestamp': None,
 'symbol': 'ETH/USDT',
 'type': 'limit',
 'timeInForce': 'GTC',
 'postOnly': False,
 'side': 'sell',
 'price': 3858.33,
 'stopPrice': None,
 'amount': 0.002,
 'cost': 0.0,
 'average': None,
 'filled': 0.0,
 'remaining': 0.002,
 'status': 'open',
 'fee': None,
 'trades': [],
 

In [58]:
# 대기 주문 조회
open_orders = binance.fetch_open_orders(
    symbol="ETH/USDT"
)
pprint.pprint(open_orders)

[{'amount': 0.002,
  'average': None,
  'clientOrderId': 'x-xcKtGhcuc3764ad988599b9c5d64a3',
  'cost': 0.0,
  'datetime': '2021-10-17T15:07:53.478Z',
  'fee': None,
  'fees': [],
  'filled': 0.0,
  'id': '8389765509101328129',
  'info': {'avgPrice': '0',
           'clientOrderId': 'x-xcKtGhcuc3764ad988599b9c5d64a3',
           'closePosition': False,
           'cumQuote': '0',
           'executedQty': '0',
           'orderId': '8389765509101328129',
           'origQty': '0.002',
           'origType': 'LIMIT',
           'positionSide': 'BOTH',
           'price': '3858.33',
           'priceProtect': False,
           'reduceOnly': False,
           'side': 'SELL',
           'status': 'NEW',
           'stopPrice': '0',
           'symbol': 'ETHUSDT',
           'time': '1634483273478',
           'timeInForce': 'GTC',
           'type': 'LIMIT',
           'updateTime': '1634483273478',
           'workingType': 'CONTRACT_PRICE'},
  'lastTradeTimestamp': None,
  'postOnly': Fal

In [59]:
# 모든 대기 주문 취소
binance.cancel_all_orders(symbol = "ETH/USDT")

{'code': '200', 'msg': 'The operation of cancel all open order is done.'}

In [98]:
# 물타기
fileName = getTodayDate() + ".txt"
with open(fileName) as f:
    lines = f.readlines()
eth15m = binance.fetch_ohlcv(
    symbol="ETH/USDT",
    timeframe='15m',
    since=None,
    limit=2)
candle = eth15m[0]
if int(lines[-1].split()[0]) > candle[0]:
    print("이전 봉에 구매 이력 있어서 물타기 스킵")
else:
    if candle[4] - candle[1] < -3:
        print("물타기 하기~")
        water()
    else:
        print("이번 봉은 음봉이 아니라 물타기 스킵")
eth15m[0]
print(time.time())
eth15m

이전 봉에 구매 이력 있어서 스킵
1634664650.381064


[[1634663700000, 3821.2, 3823.57, 3812.44, 3813.35, 10643.144],
 [1634664600000, 3813.35, 3814.0, 3808.63, 3809.93, 1806.216]]